### 引用相应的包

In [1]:
import pandas as pd  
import numpy as np
import datetime
import backtrader as bt
import matplotlib.pyplot as plt
%matplotlib inline   

In [ ]:
#正常显示画图时出现的中文和负号
from pylab import mpl    
mpl.rcParams['figure.figsize'] = 13, 8
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

### 数据准备（Data）

In [2]:
#datapath=r'D:\Baiducloud\12M8-202005\1MIN\EURUSD_M1.csv'    #windows
#datapath=r'D:\Baiducloud\12M8-202007\EURUSD_D1.csv'          #windows
#datapath=r'/Users/nilei/Nutstore/Data/EURUSD_M1.csv'         #macos
datapath=r'/Users/nilei/Nutstore/Data/EURUSD_D1.csv'         #macos
dt=pd.read_csv(datapath)
dt.rename(columns={'Time (UTC)':'Date'},inplace=True)
dt.index=pd.to_datetime(dt.Date)
dt.head()

,Date,Open,High,Low,Close,Volume
Date,,,,,,
2010-01-01,2010.01.01 00:00:00,1.43283,1.43356,1.43181,1.43335,76787.4
2010-01-03,2010.01.03 00:00:00,1.43295,1.43359,1.42910,1.43141,6331.3
2010-01-04,2010.01.04 00:00:00,1.43143,1.44556,1.42559,1.44244,80019.4
2010-01-05,2010.01.05 00:00:00,1.44238,1.44834,1.43445,1.43634,79887.1
2010-01-06,2010.01.06 00:00:00,1.43638,1.44342,1.42807,1.44005,80971.8


In [3]:
dt.drop(columns='Date',inplace=True)
#数据太大，选择合适的时间段进行回测
dt_1=dt['2015-1-1':'2019-10-3']
dt_1.head()

,Open,High,Low,Close,Volume
Date,,,,,
2015-01-01,1.21038,1.21073,1.20783,1.20869,5732.39
2015-01-02,1.20871,1.20890,1.20004,1.20017,119384.46
2015-01-04,1.19495,1.19537,1.18677,1.19469,13740.64
2015-01-05,1.19469,1.19762,1.18872,1.19374,159296.56
2015-01-06,1.19374,1.19687,1.18461,1.18722,177669.80


In [ ]:
import datetime as datetime
data = bt.feeds.PandasData(dataname=dt_1,                               
                            fromdate=datetime.datetime(2015, 10, 1),                               
                            todate=datetime.datetime(2019, 10, 1) )           #这里面要尤其注意

In [ ]:
#如果是这自己特有的数据结构，可以通过继承进行组合成个性化的类，比如：
class Addmoredata(PandasData):
    lines = ('turnover_rate','pe','pb',)
    params = (('turnover_rate',7),('pe',8),('pb',9),)       #这里面的pe，pb的数字代表的是位置，与下面的策略里面的params有所不同

### 策略模块（Strategy） 

In [ ]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 150),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        #print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        """"
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
        """

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.4f, Cost: %.4f, Comm %.4f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.4f, Cost: %.4f, Comm %.4f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.4f, NET %.4f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.4f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.4f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.4f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
# RSI策略
class MyStrategy(bt.Strategy):
    params=(('short',30),                   # RSI的下限
            ('long',70),)                   # RSI的上限

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < self.params.short:
                self.buy()
        
        else: 
            if self.rsi > self.params.long:
                self.sell()

### 分析数据（Analyzer） 

### 输出数据（Writter） 

In [ ]:
MyWriter=bt.WriterFile(
)

### 驱动模块（Cerebro） 

In [ ]:
def runstrat():    
    # Create a cerebro entity
    cerebro = bt.Cerebro(stdstats=True)

    # Add a strategy
    strats = cerebro.addstrategy(MyStrategy)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    
    # Add a writerfile
    #cerebro.addwriter(bt.WriterFile, out='/Users/nilei/Nutstore/Data/EURUSD_Result.csv',csv=True, rounding=5)
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.0)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.4f' % cerebro.broker.getvalue())
    
    # Run over everything
    ret=cerebro.run()
       
    # Print out the final result
    print('Final Portfolio Value: %.4f' % cerebro.broker.getvalue())
    
    cerebro.plot(volume=False)
    

### 运行（Run） 

In [ ]:
if __name__ == '__main__':   #在py文件里面运行需要
    runstrat()